In [ ]:
특정 커뮤니티 특정 게시판 게시물의 고유번호, 제목, 게시일자, 조회수를 크롤링합니다.
예시에서는 인스티즈(https://www.instiz.net/) 반려동물 게시판이 대상입니다.

In [1]:
import re
import json
import math
import datetime
import requests
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from numpy import nan as NA
from pandas import Series,DataFrame
import time

In [2]:
def get_instiz_title() :

    c_no=[]
    cate=[]
    title=[]
    time=[]
    m_no=[]
    for pages in range(1,10) :
        try :
            response = urllib.request.urlopen('https://www.instiz.net/name?page='+str(pages)+'&category=12')
            response_body = response.read()
            soup = BeautifulSoup(response_body.decode('utf-8'), 'html.parser')
            for nos in [soup.select('td.listno')[len(soup.select('td.notice_leaf')*4):][4*i] for i in range(19)] :
                c_no.append(nos.get_text())
            for cates in soup.select('td.minitext.listnm')[len(soup.select('td.notice_leaf')):] :
    #            cate.append(cates.get_text())
                cate.append('instiz')
            for titles in [x.select('span')[0].text for x in soup.select('td.listsubject')[len(soup.select('td.notice_leaf')):]] :
                title.append(titles)
            for times in soup.select('td.listno.regdate') :
                time.append(times.get_text())
            for m_nos in [soup.select('td.listno')[len(soup.select('td.notice_leaf')*4):][4*i+2] for i in range(19)] :
                m_no.append(m_nos.get_text())
        except : 
            pass
    
    instiz_title = pd.DataFrame({'no' : c_no, 'cate' : cate, 'title' : title, 'time' : time, 'm_no' : m_no})
    instiz_title.time = instiz_title.time.map(lambda x : str(datetime.datetime.now().year)+"."+x if len(x) > 8 else pd.to_datetime(datetime.datetime.now().strftime('%Y.%m.%d')))
    instiz_title.time = instiz_title.time.map(lambda x : pd.to_datetime(x,format="%Y.%m.%d %H:%M"))
    instiz_title = instiz_title.set_index('time')
    instiz_title = instiz_title.loc[(datetime.datetime.now()+datetime.timedelta(days=-1)).strftime('%Y-%m-%d'):(datetime.datetime.now()+datetime.timedelta(days=-8)).strftime('%Y-%m-%d')]
    print('instiz : '+str(len(instiz_title))+'개 추출')
    return instiz_title


In [3]:
instiz_df=get_instiz_title()

instiz : 33개 추출


In [4]:
instiz_df

,no,cate,title,m_no
time,,,,
2019-09-13 20:05:00,33083320,instiz,새끼강아지 하늘에 보내고 묻어주고 왔어,30
2019-09-13 19:53:00,33083187,instiz,아니 자기가 키우는 고양이들중에 배변패드에 하는 애들있어?,53
2019-09-13 19:21:00,33082883,instiz,본인표출,114
2019-09-13 17:47:00,33082002,instiz,추석이니까 한복!40,2798
2019-09-13 17:32:00,33081874,instiz,오늘 저희 아깽이가 무지개다리를 건넜어요,54
2019-09-13 16:25:00,33081237,instiz,티비로 유튜브보는데ㅋㅋㅋ 강아지가 유튜버 보고 짖는다ㅋㄱㅋ,12
2019-09-13 15:20:00,33080606,instiz,배경화면 골라죠라,56
2019-09-12 20:37:00,33071705,instiz,햄스터 잘 아는 사람?(로보로스키),59
2019-09-12 16:35:00,33068984,instiz,이거 아침에 일어날때 우리들 같지 않아? ㅋㅋㅋㅋㅋ,143
